**SDG 11: Sustainable Cities and Communities**

Categorizing Different Kinds of Trash In Manila City

This is important because, as the recent floods have demonstrated, the same trash that rises with the flood is oftentimes the same trash that caused it in the first place. Our goal is to identify different kinds of trash to pave the way for more informed decisions from policymakers going forward.

In [ ]:
from google.colab import drive

import os
import shutil
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import pandas as pd
from collections import defaultdict
import time

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks')

class AgenticFileSorter:
    """
    An agentic AI system that perceives, reasons, and acts on image files.
    """

    def __init__(self, source_dir, base_sorted_dir):
        self.source_dir = os.path.join(os.getcwd(), "FP_Source")
        self.base_sorted_dir = os.path.join(os.getcwd(), "FP_Sorted")
        self.model = None
        self.category_mapping = {
            # Common garbage/recyclable items mapping
            'bottle': 'Bottles',
            'can': 'Cans',
            'tire': 'Tires',
            'bin': 'Bins',
            'trash': 'General_Waste',
            'bag': 'Bags',
            'container': 'Containers',
            'cup': 'Cups',
            'box': 'Boxes',
            'paper': 'Paper',
            'plastic': 'Plastic_Items',
            'metal': 'Metal_Items',
            'glass': 'Glass_Items',
            'cardboard': 'Cardboard',
            'furniture': 'Furniture',
            'structure': 'Structures'
        }
        self.sorted_counts = defaultdict(int)

    def initialize_model(self):
        """Initialize the pre-trained deep learning model."""
        print("🤖 AGENT INITIALIZATION: Loading pre-trained MobileNetV2 model...")
        self.model = MobileNetV2(weights='imagenet', include_top=True)
        print("✅ Model loaded successfully!\n")

    def perceive(self, image_path):
        """
        PERCEIVE: Load and preprocess the image for analysis.
        """
        print(f"👁️  PERCEIVING: Loading image {os.path.basename(image_path)}")
        try:
            # Load and preprocess image
            img = Image.open(image_path)

            # Convert to RGB if necessary
            if img.mode != 'RGB':
                img = img.convert('RGB')

            # Resize to model input size
            img = img.resize((224, 224))

            # Convert to array and preprocess
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            print(f"   📊 Image processed: {img_array.shape}")
            return img_array, True

        except Exception as e:
            print(f"   ❌ Error processing image: {str(e)}")
            return None, False

    def reason(self, processed_image, image_name):
        """
        REASON: Analyze the image content and determine appropriate category.
        """
        print(f"🧠 REASONING: Analyzing content of {image_name}")

        try:
            # Get predictions from the model
            predictions = self.model.predict(processed_image, verbose=0)
            decoded_predictions = decode_predictions(predictions, top=5)[0]

            print("   🔍 Top predictions:")
            for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
                print(f"      {i+1}. {label}: {score:.4f}")

            # Determine category based on predictions
            category = self._map_to_category(decoded_predictions)
            confidence = decoded_predictions[0][2]  # Top prediction confidence

            print(f"   💡 REASONING RESULT: Classified as '{category}' (confidence: {confidence:.4f})")
            return category, confidence

        except Exception as e:
            print(f"   ❌ Error during reasoning: {str(e)}")
            return "Unknown", 0.0

    def _map_to_category(self, predictions):
        """
        Map ImageNet predictions to garbage/recyclable categories.
        """
        # Check each prediction for relevant keywords
        for _, label, score in predictions:
            label_lower = label.lower()

            # Direct mapping checks
            for keyword, category in self.category_mapping.items():
                if keyword in label_lower:
                    return category

            # Additional specific mappings
            if any(word in label_lower for word in ['bottle', 'flask', 'jug']):
                return 'Bottles'
            elif any(word in label_lower for word in ['can', 'tin']):
                return 'Cans'
            elif any(word in label_lower for word in ['tire', 'tyre', 'wheel']):
                return 'Tires'
            elif any(word in label_lower for word in ['bin', 'basket', 'container']):
                return 'Bins'
            elif any(word in label_lower for word in ['bag', 'sack']):
                return 'Bags'
            elif any(word in label_lower for word in ['cup', 'mug', 'glass']):
                return 'Cups'
            elif any(word in label_lower for word in ['box', 'carton']):
                return 'Boxes'
            elif any(word in label_lower for word in ['chair', 'table', 'furniture']):
                return 'Furniture'
            elif any(word in label_lower for word in ['building', 'structure', 'wall']):
                return 'Structures'

        # If no specific category found, use general classification
        top_prediction = predictions[0][1].lower()
        if any(word in top_prediction for word in ['plastic', 'synthetic']):
            return 'Plastic_Items'
        elif any(word in top_prediction for word in ['metal', 'steel', 'iron']):
            return 'Metal_Items'
        elif any(word in top_prediction for word in ['paper', 'book', 'magazine']):
            return 'Paper'
        else:
            return 'General_Items'

    def act(self, image_path, category):
        """
        ACT: Move the image to the appropriate category folder.
        """
        image_name = os.path.basename(image_path)
        print(f"🎯 ACTING: Moving {image_name} to {category} folder")

        try:
            # Create category directory if it doesn't exist
            category_dir = os.path.join(self.base_sorted_dir, category)
            os.makedirs(category_dir, exist_ok=True)

            # Move file to category directory
            destination_path = os.path.join(category_dir, image_name)
            shutil.move(image_path, destination_path)

            # Update count
            self.sorted_counts[category] += 1

            print(f"   ✅ Successfully moved to {destination_path}")
            return True

        except Exception as e:
            print(f"   ❌ Error during action: {str(e)}")
            return False

    def process_all_images(self):
        """
        Main processing loop: perceive, reason, and act on all images.
        """
        print("🚀 STARTING AGENTIC FILE SORTING PROCESS")
        print("=" * 60)

        # Initialize model
        self.initialize_model()

        # Get list of image files
        image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif'}
        image_files = [f for f in os.listdir(self.source_dir)
                      if os.path.splitext(f.lower())[1] in image_extensions]

        print(f"📁 SOURCE DIRECTORY: {self.source_dir}")
        print(f"🎯 DESTINATION DIRECTORY: {self.base_sorted_dir}")
        print(f"📷 FOUND {len(image_files)} IMAGES TO PROCESS")
        print("\nInitial file listing:")
        for i, filename in enumerate(image_files, 1):
            print(f"   {i}. {filename}")
        print("\n" + "=" * 60)

        # Process each image
        processed_count = 0
        for i, filename in enumerate(image_files, 1):
            print(f"\n🔄 PROCESSING IMAGE {i}/{len(image_files)}")
            print("-" * 40)

            image_path = os.path.join(self.source_dir, filename)

            # PERCEIVE
            processed_image, success = self.perceive(image_path)
            if not success:
                print(f"   ⏭️  Skipping {filename} due to perception error")
                continue

            # REASON
            category, confidence = self.reason(processed_image, filename)

            # ACT
            if self.act(image_path, category):
                processed_count += 1

            print(f"   📊 Processing complete for {filename}")
            time.sleep(0.5)  # Small delay for readability

        print("\n" + "=" * 60)
        print("🎉 PROCESSING COMPLETE!")
        print(f"✅ Successfully processed {processed_count}/{len(image_files)} images")

        # Display summary
        self.display_summary()

        # Show final directory state
        self.show_final_directories()

    def display_summary(self):
        """
        Display a summary table of the sorting results.
        """
        print("\n📊 SORTING SUMMARY TABLE")
        print("=" * 50)

        if self.sorted_counts:
            # Create DataFrame for better formatting
            summary_data = []
            total_files = 0

            for category, count in sorted(self.sorted_counts.items()):
                summary_data.append({
                    'Category': category,
                    'Files Sorted': count,
                    'Percentage': f"{(count/sum(self.sorted_counts.values()))*100:.1f}%"
                })
                total_files += count

            df = pd.DataFrame(summary_data)
            print(df.to_string(index=False))
            print(f"\nTOTAL FILES PROCESSED: {total_files}")
        else:
            print("No files were successfully processed.")

        print("=" * 50)

    def show_final_directories(self):
        """
        Show the final state of source and destination directories.
        """
        print("\n📁 FINAL DIRECTORY STATE")
        print("=" * 50)

        # Show source directory (should be empty or contain only non-image files)
        print(f"\n📂 SOURCE DIRECTORY: {self.source_dir}")
        try:
            remaining_files = os.listdir(self.source_dir)
            if remaining_files:
                print("   Remaining files:")
                for f in remaining_files:
                    print(f"   - {f}")
            else:
                print("   ✅ Empty (all images successfully sorted)")
        except FileNotFoundError:
            print("   Directory not found")

        # Show destination directories
        print(f"\n📂 DESTINATION BASE: {self.base_sorted_dir}")
        try:
            if os.path.exists(self.base_sorted_dir):
                categories = os.listdir(self.base_sorted_dir)
                if categories:
                    for category in sorted(categories):
                        category_path = os.path.join(self.base_sorted_dir, category)
                        if os.path.isdir(category_path):
                            files = os.listdir(category_path)
                            print(f"   📁 {category}/ ({len(files)} files)")
                            for f in files:
                                print(f"      - {f}")
                else:
                    print("   No categories created")
            else:
                print("   Directory not created")
        except Exception as e:
            print(f"   Error reading directory: {e}")

# CONFIGURATION
SOURCE_DIRECTORY = os.path.join(os.getcwd(), "unsorted_images")
BASE_SORTED_DIR = os.path.join(os.getcwd(), "sorted_images")

# MAIN EXECUTION
if __name__ == "__main__":
    print("🤖 AGENTIC FILE SORTER WITH IMAGE CLASSIFICATION")
    print("=" * 60)
    print("This system demonstrates AI agents that:")
    print("👁️  PERCEIVE: Load and process digital images")
    print("🧠 REASON: Classify content using deep learning")
    print("🎯 ACT: Sort files into appropriate categories")
    print("=" * 60)

    # Check if source directory exists
    if not os.path.exists(SOURCE_DIRECTORY):
        print(f"❌ ERROR: Source directory '{SOURCE_DIRECTORY}' not found!")
        print(f"Please create the directory and add your 15 images.")
        print(f"Expected path: {SOURCE_DIRECTORY}")
    else:
        # Create and run the agentic sorter
        sorter = AgenticFileSorter(SOURCE_DIRECTORY, BASE_SORTED_DIR)
        sorter.process_all_images()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🤖 AGENTIC FILE SORTER WITH IMAGE CLASSIFICATION
This system demonstrates AI agents that:
👁️  PERCEIVE: Load and process digital images
🧠 REASON: Classify content using deep learning
🎯 ACT: Sort files into appropriate categories
🚀 STARTING AGENTIC FILE SORTING PROCESS
🤖 AGENT INITIALIZATION: Loading pre-trained MobileNetV2 model...
14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ Model loaded successfully!

📁 SOURCE DIRECTORY: /content/drive/MyDrive/Colab Notebooks/FP_Source
🎯 DESTINATION DIRECTORY: /content/drive/MyDrive/Colab Notebooks/FP_Sorted
📷 FOUND 47 IMAGES TO PROCESS

Initial file listing:
   1. 523939307_122100073664958305_2036230428971117366_n.jpg
   2. 524317373_122100073778958305_8057656282575537221_n.jpg
   3. 524743367_122100073802958305_5993761458111957552_n.jpg
   4. 524860724_122100073736958305_6621883586239636640_n.jpg
   5. 523310082_12